In [26]:
import pandas as pd
import json
import numpy as np
from scipy import stats

In [2]:
with open('../data/letter_frequency.json', 'r') as file:
    frequency = json.load(file)
    frequency = pd.Series(frequency).to_frame()
    frequency['letter'] = frequency.index
    frequency['prob'] = frequency[0] / 100
    frequency.index = np.arange(0, len(frequency))
    frequency = frequency.drop(columns=[0])

In [3]:
frequency

,letter,prob
0,a,0.08167
1,b,0.01492
2,c,0.02782
3,d,0.04253
4,e,0.12702
5,f,0.02228
6,g,0.02015
7,h,0.06094
8,i,0.06966
9,j,0.00153


In [54]:
cipher = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
ascii_cipher = bytearray.fromhex(cipher).decode()
unique_letters = ''.join(set(ascii_cipher))
arr_cipher = np.array([ascii_cipher.count(c) / len(ascii_cipher) for c in unique_letters])

In [182]:
np.argsort(arr_cipher)[::-1], np.sort(arr_cipher)[::-1]

(array([15,  3, 12,  9, 10,  0, 14, 13, 11, 17,  8,  7,  6,  5,  4, 16,  2,
         1, 18], dtype=int64),
 array([0.17647059, 0.14705882, 0.08823529, 0.05882353, 0.05882353,
        0.05882353, 0.05882353, 0.02941176, 0.02941176, 0.02941176,
        0.02941176, 0.02941176, 0.02941176, 0.02941176, 0.02941176,
        0.02941176, 0.02941176, 0.02941176, 0.02941176]))

In [180]:
unique_letters[15]

'x'

In [21]:
def find_letter(letter_prob, df):
    prob_arr = np.array(df.prob)
    argmin_distance = np.argmin(np.abs(letter_prob - prob_arr))
    return df.iloc[argmin_distance]

In [138]:
abc_prob = np.array(frequency.prob)
cipher_freq_matrix = np.tile(arr_cipher, (len(abc_prob), 1))
distances = np.abs(cipher_freq_matrix.T - abc_prob)
z_scores = (distances.min(axis=1) - distances.mean(axis=1)) / distances.std(axis=1)
p_values = stats.norm.cdf(z_scores)

In [185]:
closest_letters = np.argsort(distances)[:, :5] + 65

In [187]:
np.sort(distances)[:, :5]

array([[0.00104647, 0.00211647, 0.00444647, 0.00866647, 0.01083647],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.02003882, 0.05649882, 0.06538882, 0.07198882, 0.07739882],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00104647, 0.00211647, 0.00444647, 0.00866647, 0.01083647],
       [0.00104647, 0.00211647, 0.00444647, 0.00866647, 0.01083647],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00232471, 0.00656529, 0.01316529, 0.01857529, 0.02074529],
       [0.00159176, 0.00183176, 0.00535176, 0.00581176, 0.00713176],
       [0.00104647, 0.00211647, 0.

In [184]:
for idx, row in enumerate(closest_letters):
    keys = []
    for jdx, letter in enumerate(row):
        keys.append(chr(letter ^ ord(unique_letters[idx])))
    print(idx, keys)

0 ['k', 'q', 'j', 'w', 'p']
1 ['w', 'a', 'y', 'c', 'r']
2 ['x', 'n', 'v', 'l', '}']
3 ['r', 'c', 'v', 'x', '~']
4 ['V', '@', 'X', 'B', 'S']
5 ['n', 'x', '`', 'z', 'k']
6 ['|', 'j', 'r', 'h', 'y']
7 ['<', '*', '2', '(', '9']
8 ['k', '}', 'e', '\x7f', 'n']
9 ['c', 'y', 'b', '\x7f', 'x']
10 ['I', 'S', 'H', 'U', 'R']
11 ['}', 'k', 's', 'i', 'x']
12 ['b', 'w', 'y', '\x7f', 'x']
13 ['~', 'h', 'p', 'j', '{']
14 ['a', '{', '`', '}', 'z']
15 ['=', ',', '9', '7', '1']
16 ['y', 'o', 'w', 'm', '|']
17 ['\x7f', 'i', 'q', 'k', 'z']
18 ['h', '~', 'f', '|', 'm']


In [ ]:
for letter in ascii_cipher:
    print()